In [1]:
pwd

u'C:\\Users\\sbardelp\\Documents\\Final Project'

In [2]:
#Work4WalmarT
#TripType - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category.
#VisitNumber - an id corresponding to a single trip by a single customer
#Weekday - the weekday of the trip
#Upc - the UPC number of the product purchased
#ScanCount - the number of the given item that was purchased. A negative value indicates a product return.
#DepartmentDescription - a high-level description of the item's department
#FinelineNumber - a more refined category for each of the products, created by Walmart
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm







col_names = ['TripType', 'VisitNumber', 'Weekday', 'upc','ScanCount', 'DepartmentDescription','FinelineNumber']
walmart = pd.read_csv('train.csv', names=col_names, skiprows=1)

In [3]:
walmart['departmentnumber'] = walmart.DepartmentDescription.map({'1-HR PHOTO':1,
'ACCESSORIES':2,
'AUTOMOTIVE':3,
'BAKERY':4,
'BATH AND SHOWER':5,
'BEAUTY':6,
'BEDDING':7,
'BOOKS AND MAGAZINES':8,
'BOYS WEAR':9,
'BRAS & SHAPEWEAR':10,
'CAMERAS AND SUPPLIES':11,
'CANDY, TOBACCO, COOKIES':12,
'CELEBRATION':13,
'COMM BREAD':14,
'CONCEPT STORES':15,
'COOK AND DINE':16,
'DAIRY':17,
'DSD GROCERY':18,
'ELECTRONICS':19,
'FABRICS AND CRAFTS':20,
'FINANCIAL SERVICES':21,
'FROZEN FOODS':22,
'FURNITURE':23,
'GIRLS WEAR, 4-6X  AND 7-14':24,
'GROCERY DRY GOODS':25,
'HARDWARE':26,
'HEALTH AND BEAUTY AIDS':27,
'HOME DECOR':28,
'HOME MANAGEMENT':29,
'HORTICULTURE AND ACCESS':30,
'HOUSEHOLD CHEMICALS/SUPP':31,
'HOUSEHOLD PAPER GOODS':32,
'IMPULSE MERCHANDISE':33,
'INFANT APPAREL':34,
'INFANT CONSUMABLE HARDLINES':35,
'JEWELRY AND SUNGLASSES':36,
'LADIES SOCKS':37,
'LADIESWEAR':38,
'LARGE HOUSEHOLD GOODS':39,
'LAWN AND GARDEN':40,
'LIQUOR,WINE,BEER':41,
'MEAT - FRESH & FROZEN':42,
'MEDIA AND GAMING':43,
'MENS WEAR':44,
'MENSWEAR':45,
'NULL':46,
'OFFICE SUPPLIES':47,
'OPTICAL - FRAMES':48,
'OPTICAL - LENSES':49,
'OTHER DEPARTMENTS':50,
'PAINT AND ACCESSORIES':51,
'PERSONAL CARE':52,
'PETS AND SUPPLIES':53,
'PHARMACY OTC':54,
'PHARMACY RX':55,
'PLAYERS AND ELECTRONICS':56,
'PLUS AND MATERNITY':57,
'PRE PACKED DELI':58,
'PRODUCE':59,
'SEAFOOD':60,
'SEASONAL':61,
'SERVICE DELI':62,
'SHEER HOSIERY':63,
'SHOES':64,
'SLEEPWEAR/FOUNDATIONS':65,
'SPORTING GOODS':66,
'SWIMWEAR/OUTERWEAR':67,
'TOYS':68,
'WIRELESS':69,
})


In [4]:
walmart['Weekdaynumber'] = walmart.Weekday.map ({'Sunday':1,
'Monday':2,
'Tuesday':3,
'Wednesday':4,
'Thursday':5,
'Friday':6,
'Saturday':7,
})


In [5]:
# WEEKDAY  dummy variables and multiply them by Scancount
#weekdaydummies = pd.get_dummies(walmart.Weekdaynumber, prefix='wd')
#weekdaydummies = weekdaydummies.multiply(walmart["ScanCount"], axis ='index')

In [6]:
# DEPARTMENT  dummy variables and multiply them by Scancount
departmentdummies = pd.get_dummies(walmart.departmentnumber, prefix='dept')
departmentdummies = departmentdummies.multiply(walmart["ScanCount"], axis ='index')
# FINELINE  dummy variables and multiply them by Scancount
#finelinedummies = pd.get_dummies(walmart.FinelineNumber, prefix='fl')
#finelinedummies = finelinedummies.multiply(walmart["ScanCount"], axis ='index')


#upcdummies = pd.get_dummies(walmart.upc, prefix = 'u')
#dask or graphlab s frame

# concatenate dummy data to walmart dataframe (axis=0 for rows, axis=1 for columns)
walmart = walmart.drop(['upc','ScanCount','Weekday','DepartmentDescription','FinelineNumber','departmentnumber'], axis=1) 
walmart = pd.concat([walmart,departmentdummies], axis=1)
walmart = walmart.groupby(['TripType','VisitNumber','Weekdaynumber'], as_index=False).sum()
walmart.head()

,TripType,VisitNumber,Weekdaynumber,dept_1.0,dept_2.0,dept_3.0,dept_4.0,dept_5.0,dept_6.0,dept_7.0,...,dept_60.0,dept_61.0,dept_62.0,dept_63.0,dept_64.0,dept_65.0,dept_66.0,dept_67.0,dept_68.0,dept_69.0
0,3,106,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,121,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,153,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,162,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,164,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = walmart.iloc[:, 2:71]
y = walmart.TripType
print X.shape

(95674, 69)


In [8]:
# **Step 1:** Import the class you plan to use
from sklearn.neighbors import KNeighborsClassifier
# **Step 2:** "Instantiate" the "estimator"
# - "Estimator" is scikit-learn's term for "model"
# - "Instantiate" means "make an instance of"
knn = KNeighborsClassifier(n_neighbors=1)

# - Name of the object does not matter
# - Can specify tuning parameters (aka "hyperparameters") during this step
# - All parameters not specified are set to their defaults
print knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=1, p=2, weights='uniform')


In [9]:
knn.fit(X, y)
y_pred = knn.predict(X)
from sklearn import metrics
print metrics.accuracy_score(y, y_pred)

0.888872629973


In [10]:
# ### Understanding the `train_test_split` function
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# before splitting
print X.shape

# after splitting
print X_train.shape
print X_test.shape

# before splitting
print y.shape

# after splitting
print y_train.shape
print y_test.shape

(95674, 69)
(57404, 69)
(38270, 69)
(95674L,)
(57404L,)
(38270L,)
